In [15]:
import csv
from tqdm import tqdm
import traceback as tb
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import polars as pl
# import heartrate; heartrate.trace(browser=True, daemon=True)

columns = ["snapshot_date", "total_off_site", "total", "convicted_or_sentenced", "detained_or_awaiting_trial", "other_offense", "id"]
snap_dates = ["2021-04-01", "2021-05-01", "2021-06-01", "2021-07-01", "2021-08-01", "2021-09-01", "2021-10-01", "2021-11-01", "2021-12-01", "2022-02-01", "2022-03-01", "2022-04-01"]

In [49]:
file = "Albany County Jail.csv"


df = pl.read_csv(f"./files/{file}")
# print(df)
# Drop last column (date range)
df = df.drop(['jail', 'delete'])
# Remove old header
df = df[1:]


In [50]:

df = df.transpose(include_header=True)
df = df.drop(["column_0", "column_2"])
print(df)
df.columns = ["snapshot_date", "total_off_site", "total", "convicted_or_sentenced", "civil_offense", "federal_offense", "technical_parole_violators", "state_readies", "detained_or_awaiting_trial"]
df = df[1:]
# print(df)
df2 = df.to_pandas()
con_col = ["state_readies", "detained_or_awaiting_trial"]
# df2[con_col] = pd.to_numeric(df2[con_col].stack(), downcast="integer", errors='coerce').unstack()
df2["state_readies"] = pd.to_numeric(df2["state_readies"], downcast="integer")
df2["detained_or_awaiting_trial"] = pd.to_numeric(df2["detained_or_awaiting_trial"], downcast="integer")

shape: (14, 9)
┌────────────┬─────────────┬──────────┬─────────────┬─────┬───────────┬───────────────────┬─────────┬──────────────────┐
│ column     ┆ column_1    ┆ column_3 ┆ column_4    ┆ ... ┆ column_6  ┆ column_7          ┆ column_ ┆ column_9         │
│ ---        ┆ ---         ┆ ---      ┆ ---         ┆     ┆ ---       ┆ ---               ┆ 8       ┆ ---              │
│ str        ┆ str         ┆ str      ┆ str         ┆     ┆ str       ┆ str               ┆ ---     ┆ str              │
│            ┆             ┆          ┆             ┆     ┆           ┆                   ┆ str     ┆                  │
╞════════════╪═════════════╪══════════╪═════════════╪═════╪═══════════╪═══════════════════╪═════════╪══════════════════╡
│ type       ┆ Boarded Out ┆ In House ┆ - Sentenced ┆ ... ┆ - Federal ┆ - Technical       ┆ - State ┆ - Other          │
│            ┆             ┆          ┆             ┆     ┆           ┆ Parole Violators  ┆ Readies ┆ Unsentenced **   │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌

NameError: name 'pa' is not defined

In [ ]:
df2["detained_or_awaiting_trial"] = df2[["detained_or_awaiting_trial", "state_readies"]].sum(axis=1)
df = pl.from_pandas(df2)
df = df.drop("state_readies")
df = df.to_pandas()
print(id)
# df["id"] = id
print(df)


df.to_csv("extracted_data", mode="a")